# El corrido de Gregorio Cortez

## Mapping the manhunt of Rio Grande border folk hero Gregorio Cortez:  the largest manhunt in U.S. history.

### June 14, 1901 to June 22, 1901

### based on lyrics from the Corrido de Gregorio Cortez (variant x english translation by Americo Paredes)

#### project author Diane López 

In [1]:
import geopandas as gpd
from pyproj import CRS
import pandas as pd
import numpy as np

from geograpy import places
import re

import geopy
from geopy.geocoders import ArcGIS
#from geopy.extra.rate_limiter import RateLimiter

import shapely
from shapely.geometry import Point
from shapely.wkt import loads
import plotly.express as px 

import contextily as cx
import xyzservices.providers as xyz
import matplotlib.pyplot as plt #to make sure there are no errors when plotting a graph
import pyproj

#import nltk

import spacy
from spacy import displacy

import googlemaps

import locationtagger
nlp = spacy.load("en_core_web_sm")

In [2]:
text = "corrido corpus\ElCorridodeGregorioCortez_X.txt"
with open(text, 'r', encoding='utf-8') as c:
    text = c.read()
    
def clean_text(text):
    cleaned= re.sub(r'[":;,.“”]', "", text)
    return(cleaned)
text = clean_text(text)
#print(text)


TxGPE=[]
nlp = spacy.load("tx_trained_ner")
doc =nlp(text) 
#print(doc)
for ent in doc.ents:
    #print(ent.text, ent.label_)
    if ent.label_ == "GPE":
        TxGPE.append(ent.text)
print(TxGPE)

['Karnes', 'Gonzales', 'Belmont', 'Laredo', 'Encinal']


In [3]:
ents = [(e.text, e.start_char, e.end_char, e.label_)for e in doc.ents]
print(ents)

ents = [(e.text)for e in doc.ents]
print(ents)

[('Karnes', 18, 24, 'GPE'), ('Gonzales', 1051, 1059, 'GPE'), ('Belmont', 1140, 1147, 'GPE'), ('Laredo', 1550, 1556, 'GPE'), ('Encinal', 1864, 1871, 'GPE')]
['Karnes', 'Gonzales', 'Belmont', 'Laredo', 'Encinal']


In [4]:
displacy.render(doc, style ='ent', jupyter=True, page=True)

Geoparsing: Finding places from the corrido lyrics

In [5]:
df = pd.DataFrame(ents, columns=["NER_Places"])
geolocator = ArcGIS(user_agent='CorridosMap')
geocode = lambda query: geolocator.geocode("%s, Texas" % query)
df['Location'] = df['NER_Places'].apply(geocode)
#df['geometry'] = df['NER_Places'].apply(geocode)
df

,NER_Places,Location
0,Karnes,"(Karnes County, Texas, (28.905630820000056, -9..."
1,Gonzales,"(Gonzales, Texas, (29.500150000000076, -97.452..."
2,Belmont,"(Belmont, Texas, (29.52287000000007, -97.68328..."
3,Laredo,"(Laredo, Texas, (27.530920000000037, -99.50230..."
4,Encinal,"(Encinal, Texas, (28.046230000000037, -99.3566..."


In [6]:
gdf = gpd.tools.geocode(df.Location, provider='ArcGIS')
gdf

,geometry,address
0,POINT (-97.85942 28.90563),"Karnes County, Texas"
1,POINT (-97.45223 29.50015),"Gonzales, Texas"
2,POINT (-97.68329 29.52287),"Belmont, Texas"
3,POINT (-99.50231 27.53092),"Laredo, Texas"
4,POINT (-99.35670 28.04623),"Encinal, Texas"


In [8]:
px.set_mapbox_access_token(open("mapboxtoken").read())
fig = px.line_mapbox(gdf,
                        lat=gdf.geometry.y,
                        lon=gdf.geometry.x,
                        hover_name="address",
                       
                        )

fig.update_layout(mapbox_style="stamen-terrain", mapbox_zoom=6, mapbox_center_lat = 29,
    margin={"r":0,"t":0,"l":0,"b":0})



fig.show()

In [9]:
gdf.to_json()

'{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"address": "Karnes County, Texas"}, "geometry": {"type": "Point", "coordinates": [-97.85942102299998, 28.905630820000056]}}, {"id": "1", "type": "Feature", "properties": {"address": "Gonzales, Texas"}, "geometry": {"type": "Point", "coordinates": [-97.45222999999999, 29.500150000000076]}}, {"id": "2", "type": "Feature", "properties": {"address": "Belmont, Texas"}, "geometry": {"type": "Point", "coordinates": [-97.68328999999994, 29.52287000000007]}}, {"id": "3", "type": "Feature", "properties": {"address": "Laredo, Texas"}, "geometry": {"type": "Point", "coordinates": [-99.50230999999997, 27.530920000000037]}}, {"id": "4", "type": "Feature", "properties": {"address": "Encinal, Texas"}, "geometry": {"type": "Point", "coordinates": [-99.35669999999999, 28.046230000000037]}}]}'